In [113]:
import pandas as pd
import numpy as np

# combined data - 2 tables

In [114]:
path_tables = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 2 tables/"

In [115]:
filename_careers = "careers.xlsx"

careers = pd.read_excel(path_tables + filename_careers,dtype="str")
careers.shape

(9001, 12)

In [116]:
careers[careers["CareerString"].isna()]

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,CareerSubstring,OrgString,PrimaryInstitution,OrgName,Position,Notes


In [117]:
filename_leadercareerlink = "leadercareerlink.xlsx"

In [118]:
filename_leaderjoblink = "leaderjoblink.xlsx"

In [119]:
filename_joborglink = "joborglink.xlsx"

# combined data - 1 cleaning/cleaning step 2 - career/

In [120]:
path_cleaning = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 2 - career/"

In [121]:
subpath_2_1 = "2.1 career_undivided_unparsed_uncoded/"
subpath_2_2 = "2.2 career_divided_unparsed_uncoded/"
subpath_2_3 = "2.3 joborglink/"
subpath_2_4 = "2.4 orgtree/"
subpath_2_5 = "2.5 position/"
subpath_2_6 = "2.6 career_reassembled/"
subpath_2_7 = "2.7 orgtree position & rank/"

In [122]:
# 1.0 orgtree_position_rank

# constructing new file from:
## 2022 북한_기관별_인명록_북한정보포털 게재용
## orgtree table

filein_opr = "1.0 orgtree_position_rank.xlsx"
opr = pd.read_excel(path_cleaning + subpath_2_7 + filein_opr,dtype="str")
opr.shape

(2270, 22)

In [123]:
opr.head(2)

,InsideGov,InstitutionType,OrgType,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_PI,Change_OrgName,Alias_OrgName
0,1,노동당,NaN,노동당,NaN,1.0.0.0.0,0,총비서,NaN,NaN,Current,Current,Current,NaN,1,0,0,0,0,NaN,NaN,NaN
1,1,노동당,NaN,노동당,당중앙위원회,1.1.0.0.0,1,NaN,NaN,"위원, 후보위원",Current,Current,Current,NaN,1,1,0,0,0,NaN,NaN,NaN


In [124]:
# orgtree_copy

# copy of orgtree table

filein_ot = "orgtree_copy.xlsx"
ot = pd.read_excel(path_cleaning + subpath_2_7 + filein_ot,dtype="str")
ot.shape

(1406, 19)

In [125]:
# position_rank_jacob_2 - hand-coded

# Jacob's hand_ranked positions of 내각, 정무원

# filein_posrank_admin = "position_rank_jacob_2.xlsx"
# pr_admin = pd.read_excel(path_cleaning + subpath_2_7 + filein_posrank_admin,dtype="str")
# pr_admin.shape

In [126]:
# position_rank_esther - original

# original positions of 노동당, before hand-coding

# filein_posrank_party_orig = "position_rank_esther.xlsx"
# pr_party_orig = pd.read_excel(path_cleaning + subpath_2_7 + filein_posrank_party_orig,dtype="str")
# pr_party_orig.shape

In [127]:
# position_rank_esther_handcoded

# Esther's hand_ranked positions of 노동당

# filein_posrank_party = "position_rank_esther_handcoded.xlsx"
# pr_party = pd.read_excel(path_cleaning + subpath_2_7 + filein_posrank_party,dtype="str")
# pr_party.shape

In [128]:
# position_rank_jeongsu_handcoded

# Jeongsu's hand-coded positions of all orgs minus 노동당, 내각, 정무원

filein_posrank_else = "position_rank_jeongsu_handcoded.xlsx"
pr_else = pd.read_excel(path_cleaning + subpath_2_7 + filein_posrank_else,dtype="str")
pr_else.shape

(839, 10)

# functions: format orgtree

In [129]:
# using this on (PI,OrgName) will ensure unique & non-null keys
# using this on a larger df will ensure unique rows and non-null keys, but not unique keys

def unique_non_null_rows(olddf):
    
    df = olddf.copy()
    
    
    ### drop duplicates
    df = df.drop_duplicates(keep="first",ignore_index=True)

    ### drop null rows
    df.dropna(how="all",axis=0,inplace=True)
    
    # drop rows with null PI
    df = df[~(df["PrimaryInstitution"].isna())]
        
    ### drop rows where PI contains stop words
    stop_words_lower = ["uncertain","current","deprecated","please_revise"]
    df = df[ ~ df["PrimaryInstitution"].str.lower().isin(stop_words_lower)]
            

    print("\nUnique Non-null Rows...")
    print("")
    print("\tNon-unique rows:",olddf.shape)
    print("\tUnique rows    :",df.shape)

    ### sort
    df = df.sort_values(["PrimaryInstitution","OrgName"])
    
    return df

In [130]:
def verify_unique_rows(df):
    
    key_columns = ["PrimaryInstitution","OrgName"]
    print("\nVerifying Unique Rows...")
    print("")
    print("\tDuplicate Rows:",df[df.duplicated(keep=False)].shape)
    print("\tDuplicate Keys:",df[df.duplicated(key_columns,keep=False)].shape)
    print("\tNull Rows     :",df[df["PrimaryInstitution"].isna() & df["OrgName"].isna()].shape)

In [131]:
def merge_results(m):
    
    print("\nMerge Results...")
    print("")
    print("\tshape     :",m.shape)
    print("\tleft_only :",m[m["_merge"]=="left_only"].shape)
    print("\tboth      :",m[m["_merge"]=="both"].shape)
    print("\tright_only:",m[m["_merge"]=="right_only"].shape)

# 0. opr: format indices & OrgRank

In [132]:
opr.head(2)

,InsideGov,InstitutionType,OrgType,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_PI,Change_OrgName,Alias_OrgName
0,1,노동당,NaN,노동당,NaN,1.0.0.0.0,0,총비서,NaN,NaN,Current,Current,Current,NaN,1,0,0,0,0,NaN,NaN,NaN
1,1,노동당,NaN,노동당,당중앙위원회,1.1.0.0.0,1,NaN,NaN,"위원, 후보위원",Current,Current,Current,NaN,1,1,0,0,0,NaN,NaN,NaN


In [133]:
def update_opr_index(opr,max_level):
    
    # create and clear out existing Level Indicies
    opr["OrgRank"]=np.nan
    opr["L1_Index"]=np.nan
    opr["L2_Index"]=np.nan
    opr["L3_Index"]=np.nan
    opr["L4_Index"]=np.nan
    opr["L5_Index"]=np.nan
    
    # pad PI_Index with missing 0s up to the deepest level (4 or 5?)
    opr["PI_Index"] = opr.apply(lambda x: str(x["PI_Index"]) + ((max_level-1)-str(x["PI_Index"]).count(".")) * ".0",axis=1)
    
    # calculate OrgRank
    opr["OrgRank"] = opr.apply(lambda x: (max_level-1) - str(x["PI_Index"]).count(".0"),axis=1)
    
    # populate Level Indices using PI_Index
    opr.loc[opr["PI_Index"].notna(),"L1_Index"] = opr.loc[opr["PI_Index"].notna()].apply(lambda x: str(x["PI_Index"]).split(".")[0],axis=1)
    opr.loc[opr["PI_Index"].notna(),"L2_Index"] = opr.loc[opr["PI_Index"].notna()].apply(lambda x: str(x["PI_Index"]).split(".")[1],axis=1)
    opr.loc[opr["PI_Index"].notna(),"L3_Index"] = opr.loc[opr["PI_Index"].notna()].apply(lambda x: str(x["PI_Index"]).split(".")[2],axis=1)
    opr.loc[opr["PI_Index"].notna(),"L4_Index"] = opr.loc[opr["PI_Index"].notna()].apply(lambda x: str(x["PI_Index"]).split(".")[3],axis=1)
    opr.loc[opr["PI_Index"].notna(),"L5_Index"] = opr.loc[opr["PI_Index"].notna()].apply(lambda x: str(x["PI_Index"]).split(".")[4],axis=1)

    # sort by Level1, Level2, Level3
    sort_columns = ["L1_Index","L2_Index","L3_Index","L4_Index","L5_Index"]
    #opr.L1_Index.astype(int,errors="ignore")
    #opr.L2_Index.astype(int,errors="ignore")
    #opr.L3_Index.astype(int,errors="ignore")
    opr.sort_values(sort_columns,inplace=True)
    
    return opr

In [60]:
opr = update_opr_index(opr,5)

In [61]:
# export opr

fileout_opr = "1.0 orgtree_position_rank.xlsx"
opr.to_excel(path_cleaning + subpath_2_7 + fileout_opr,index=False)

# 0. pr_else - clean file, add LinkToNext_

In [62]:
# position_rank_esther_handcoded: restore OrgName from position_rank_esther
# Esther changed a few OrgNames; I will restore the original OrgName, but save Esther's changes

In [134]:
pr_else.head(2)

,Unnamed: 0,InsideGov,PrimaryInstitution,OrgName,Position,Position수정,IsElected,OrgRank,PositionRank,NOTE
0,0,0,4.15문화창작단,NaN,단장,NaN,NaN,1,1,NaN
1,1,0,4.15문화창작단,NaN,부단장,NaN,NaN,1,2,NaN


In [135]:
# pr_else[pr_else.Change_OrgName.notna()].shape

In [136]:
# add LinkToNext_ fields to pr_party2
ot.head(2)

,InsideGov,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_OrgName,Alias_OrgName
0,0,4.15문화창작단,NaN,NaN,NaN,NaN,NaN,NaN,Current,Current,Current,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,6.15공동선언실천남북공동위원회북측위원회,교직원분과,NaN,NaN,NaN,NaN,NaN,Current,Current,Current,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
key_columns2 = ["PrimaryInstitution","OrgName"]
# org_columns = key_columns2 + [c for c in list(ot.columns) if "LinkToNext_" in c]
org_columns = key_columns2 + ["LinkToNext_Year"]
# print(org_columns)
pr_else2 = pr_else.merge(ot[org_columns],on=key_columns2,how="inner",indicator=False)
# merge_results(pr_else2)

In [138]:
# prep columns to merge with opr
pr_else2.rename(columns={"PositionRank":"PositionRank_manual"},inplace=True)

In [139]:
pr_else2.rename(columns={"Position수정":"Change_Position"},inplace=True)

In [140]:
pr_else2.rename(columns={"NOTE":"Notes"},inplace=True)

In [141]:
pr_else2["PositionRank_3P"]=np.nan

In [142]:
pr_else2.columns

Index(['Unnamed: 0', 'InsideGov', 'PrimaryInstitution', 'OrgName', 'Position',
       'Change_Position', 'IsElected', 'OrgRank', 'PositionRank_manual',
       'Notes', 'LinkToNext_Year', 'PositionRank_3P'],
      dtype='object')

In [143]:
reorder_columns = ['PrimaryInstitution', 'OrgName', 'LinkToNext_Year','Position', 'IsElected',
       'PositionRank_manual',  'PositionRank_3P', 'Change_Position','Notes']
pr_else2 = pr_else2[reorder_columns]

In [144]:
pr_else2.head(2)

,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,Change_Position,Notes
0,4.15문화창작단,NaN,Current,단장,NaN,1,NaN,NaN,NaN
1,4.15문화창작단,NaN,Current,부단장,NaN,2,NaN,NaN,NaN


# 1. orgtree_position_rank: update all besides 노동당, 내각, 정무원

In [145]:
# align opr with pr_else2

In [146]:
pr_else2.head(2)

,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,Change_Position,Notes
0,4.15문화창작단,NaN,Current,단장,NaN,1,NaN,NaN,NaN
1,4.15문화창작단,NaN,Current,부단장,NaN,2,NaN,NaN,NaN


In [147]:
# recursively - 

## manually update orgtree_position_rank
## open and compare with pr_party2

In [148]:
# 1.0 orgtree_position_rank

# constructing new file from:
## 2022 북한_기관별_인명록_북한정보포털 게재용
## orgtree table

filein_opr = "1.0 orgtree_position_rank.xlsx"
opr = pd.read_excel(path_cleaning + subpath_2_7 + filein_opr,dtype="str")
opr.shape

(2270, 22)

In [149]:
opr.head(2)

,InsideGov,InstitutionType,OrgType,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_PI,Change_OrgName,Alias_OrgName
0,1,노동당,NaN,노동당,NaN,1.0.0.0.0,0,총비서,NaN,NaN,Current,Current,Current,NaN,1,0,0,0,0,NaN,NaN,NaN
1,1,노동당,NaN,노동당,당중앙위원회,1.1.0.0.0,1,NaN,NaN,"위원, 후보위원",Current,Current,Current,NaN,1,1,0,0,0,NaN,NaN,NaN


In [79]:
opr = update_opr_index(opr,5)

In [80]:
# export opr

# fileout_opr = "1.0 orgtree_position_rank.xlsx"
# opr.to_excel(path_cleaning + subpath_2_7 + fileout_opr,index=False)

In [150]:
pr_else2.columns

Index(['PrimaryInstitution', 'OrgName', 'LinkToNext_Year', 'Position',
       'IsElected', 'PositionRank_manual', 'PositionRank_3P',
       'Change_Position', 'Notes'],
      dtype='object')

In [151]:
opr.columns

Index(['InsideGov', 'InstitutionType', 'OrgType', 'PrimaryInstitution',
       'OrgName', 'PI_Index', 'OrgRank', 'P1', 'P2', 'P3', 'LinkToNext_PI',
       'LinkToNext_Org', 'LinkToNext_Year', 'Notes', 'L1_Index', 'L2_Index',
       'L3_Index', 'L4_Index', 'L5_Index', 'Change_PI', 'Change_OrgName',
       'Alias_OrgName'],
      dtype='object')

In [152]:
key_columns2 = ["PrimaryInstitution","OrgName"]
pr_else3 = pr_else2.merge(opr,on=key_columns2,how="left",indicator=True,suffixes=("","_opr"))
merge_results(pr_else3)


Merge Results...

	shape     : (839, 30)
	left_only : (0, 30)
	both      : (839, 30)
	right_only: (0, 30)


In [153]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# select_columns = ["PrimaryInstitution","OrgName","PI_Index","OrgRank","Position","PositionRank_manual","P1","P2","P3","Notes","Notes_opr"] 
# pr_else3.loc[pr_else3["_merge"]=="both",select_columns]

In [154]:
pr_else3[pr_else3["_merge"]=="left_only"].head(3)

,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,Change_Position,Notes,InsideGov,InstitutionType,OrgType,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year_opr,Notes_opr,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_PI,Change_OrgName,Alias_OrgName,_merge


### Three Cases of  OrgName in NK elite career data

- Case 0: OrgName in 기관별인명록: code these from 1-199
- Case 1: OrgName not in 기관별인명록, but contained in data AND LinkToNext_Year not current: code as 500+
- Case 2: OrgName not in 기관별인명록, but contained in data AND LinkToNext_Year is current: code as 200+

### Case 0: OrgName in 기관별인명록

- Update 1.0 orgtree_position_rank with OrgName in 기관별인명록
- Code these from 1-199
- Run above routine # 1. orgtree_position_rank: update all besides 노동당, 내각, 정무원
    - to update opr & pr_else3
- Proceed to Cases 1 & 2

### Case 1: LinkToNext_Year not current - code as 500+

In [155]:
pr_else3[(pr_else3["LinkToNext_Year"]!="Current") & (pr_else3["_merge"]=="left_only")].shape

(0, 30)

In [156]:
ot[~(ot["LinkToNext_Year"]=="Current") & ~(ot["PrimaryInstitution"].isin(["노동당","내각","내각A","내각B","내각C","정무원"]))].shape

(292, 19)

In [157]:
pr_else3[(pr_else3["LinkToNext_Year"]!="Current") & (pr_else3["_merge"]=="left_only")]

,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,Change_Position,Notes,InsideGov,InstitutionType,OrgType,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year_opr,Notes_opr,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_PI,Change_OrgName,Alias_OrgName,_merge


### Case 2: Level 3 or deeper (not found in 기관별인명록) - code as 200+

In [158]:
pr_else3[pr_else3["_merge"]=="left_only"].shape

(0, 30)

In [159]:
show_columns = ["OrgName","Position","Notes","Change_OrgName","_merge"]
pr_else3.loc[pr_else3["_merge"]=="left_only",show_columns]

,OrgName,Position,Notes,Change_OrgName,_merge


In [160]:
# prep to add to opr
opr.columns

Index(['InsideGov', 'InstitutionType', 'OrgType', 'PrimaryInstitution',
       'OrgName', 'PI_Index', 'OrgRank', 'P1', 'P2', 'P3', 'LinkToNext_PI',
       'LinkToNext_Org', 'LinkToNext_Year', 'Notes', 'L1_Index', 'L2_Index',
       'L3_Index', 'L4_Index', 'L5_Index', 'Change_PI', 'Change_OrgName',
       'Alias_OrgName'],
      dtype='object')

In [161]:
pr_else3[opr.columns].head(2)

,InsideGov,InstitutionType,OrgType,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_PI,Change_OrgName,Alias_OrgName
0,0,NaN,NaN,4.15문화창작단,NaN,201.0.0.0.0,0,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,0,0,0,0,NaN,NaN,NaN
1,0,NaN,NaN,4.15문화창작단,NaN,201.0.0.0.0,0,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,0,0,0,0,NaN,NaN,NaN


In [162]:
pr_else3_left = pr_else3.loc[pr_else3["_merge"]=="left_only",opr.columns]

In [163]:
pr_else3_left.shape

(0, 22)

In [164]:
pr_else3_left.drop_duplicates(inplace=True)
pr_else3_left.shape

(0, 22)

In [165]:
pr_else3_left

,InsideGov,InstitutionType,OrgType,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_PI,Change_OrgName,Alias_OrgName


In [67]:
# export pr_else3_left to update 500+ & 200+ Series Orgs

fileout_pr_else3_left = "pr_else3_left_use and delete.xlsx"
pr_else3_left.to_excel(path_cleaning + subpath_2_7 + fileout_pr_else3_left,index=False)

# 2. pr_admin3: update PositionRank_3P and export

In [97]:
# once aligned, copy pr_party2 hand-coded PositionRank into opr

In [166]:
pr_else.shape

(839, 10)

In [167]:
pr_else2.shape

(839, 9)

In [168]:
pr_else3.shape

(839, 30)

In [169]:
pr_else3.head(10)

,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,Change_Position,Notes,InsideGov,InstitutionType,OrgType,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year_opr,Notes_opr,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_PI,Change_OrgName,Alias_OrgName,_merge
0,4.15문화창작단,NaN,Current,단장,NaN,1,NaN,NaN,NaN,0,NaN,NaN,201.0.0.0.0,0,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,0,0,0,0,NaN,NaN,NaN,both
1,4.15문화창작단,NaN,Current,부단장,NaN,2,NaN,NaN,NaN,0,NaN,NaN,201.0.0.0.0,0,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,0,0,0,0,NaN,NaN,NaN,both
2,7.7연합기업소,NaN,Current,기사장,NaN,?,NaN,NaN,NaN,0,NaN,NaN,201.0.0.0.0,0,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,0,0,0,0,NaN,NaN,NaN,both
3,강건종합군관학교,NaN,Current,교장,NaN,1,NaN,NaN,NaN,0,NaN,NaN,201.0.0.0.0,0,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,0,0,0,0,NaN,NaN,NaN,both
4,강동지구탄광연합기업소,NaN,Current,지배인,NaN,2,NaN,NaN,NaN,0,NaN,NaN,201.0.0.0.0,0,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,0,0,0,0,NaN,NaN,NaN,both
5,강서구역청산엽동농장,NaN,Current,관리위원장,NaN,2,NaN,NaN,NaN,0,NaN,NaN,201.0.0.0.0,0,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,0,0,0,0,NaN,NaN,NaN,both
6,강선제강소,NaN,Current,책임비서,NaN,1,NaN,NaN,NaN,0,NaN,NaN,201.0.0.0.0,0,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,0,0,0,0,NaN,NaN,NaN,both
7,강원도임업연합기업소,NaN,Current,지배인,NaN,2,NaN,NaN,NaN,0,NaN,NaN,201.0.0.0.0,0,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,0,0,0,0,NaN,NaN,NaN,both
8,개성무역총회사,NaN,Current,총사장,NaN,2,NaN,NaN,NaN,0,NaN,NaN,201.0.0.0.0,0,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,0,0,0,0,NaN,NaN,NaN,both
9,개성방직공장,3대혁명소조,Current,소조원,NaN,Null,NaN,NaN,NaN,0,NaN,NaN,201.201.0.0.0,1,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,201,0,0,0,NaN,NaN,NaN,both


In [170]:
def return_position_rank(x):
    
    # create list of positions in P1, P2, P3
    p_list = str(x["P1"]).split(",")+str(x["P2"]).split(",")+str(x["P3"]).split(",")
    # print(p_list)
    
    # remove "nan"
    p_list = [item.strip() for item in p_list if item.strip()!="nan"]
    # print(p_list)
    
    # list index of Position in p_list
    position_rank = np.nan
    
    if x["Position"] in p_list:
        position_rank = p_list.index(x["Position"]) + 1
    
    
    return position_rank

In [171]:
pr_else3.apply(lambda x: return_position_rank(x),axis=1)

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     3.0
29     NaN
30     4.0
31     1.0
32     3.0
33     NaN
34     NaN
35     NaN
36     NaN
37     NaN
38     NaN
39     NaN
40     1.0
41     1.0
42     NaN
43     NaN
44     NaN
45     1.0
46     NaN
47     NaN
48     1.0
49     1.0
50     NaN
51     NaN
52     NaN
53     NaN
54     NaN
55     NaN
56     NaN
57     NaN
58     NaN
59     NaN
60     NaN
61     2.0
62     2.0
63     1.0
64     NaN
65     NaN
66     2.0
67     1.0
68     NaN
69     NaN
70     NaN
71     NaN
72     2.0
73     1.0
74     NaN
75     NaN
76     NaN
77     2.0
78     1.0
79     NaN
80     NaN
81     NaN
82     NaN
83     NaN
84     NaN
85     NaN
86     NaN
87     NaN
88     NaN
89     NaN
90     NaN

In [172]:
pr_else3["PositionRank_3P"] = pr_else3.apply(lambda x: return_position_rank(x),axis=1)

In [173]:
# check
pr_else3[(pr_else3["Position"]=="위원") & (pr_else3["OrgName"]=="당중앙위원회")]

,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,Change_Position,Notes,InsideGov,InstitutionType,OrgType,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year_opr,Notes_opr,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_PI,Change_OrgName,Alias_OrgName,_merge


In [174]:
# copy Change_OrgName_opr to Change_OrgName
# copy Notes_opr to Notes
pr_else3[pr_else3["Change_OrgName"].notna()].shape

(97, 30)

In [176]:
pr_else3.head(2)

,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,Change_Position,Notes,InsideGov,InstitutionType,OrgType,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year_opr,Notes_opr,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_PI,Change_OrgName,Alias_OrgName,_merge
0,4.15문화창작단,NaN,Current,단장,NaN,1,NaN,NaN,NaN,0,NaN,NaN,201.0.0.0.0,0,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,0,0,0,0,NaN,NaN,NaN,both
1,4.15문화창작단,NaN,Current,부단장,NaN,2,NaN,NaN,NaN,0,NaN,NaN,201.0.0.0.0,0,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,0,0,0,0,NaN,NaN,NaN,both


In [177]:
# pr_else3 - how many empty PositionRank_manual & PositionRank_3P
pr_else3[pr_else3["PositionRank_manual"].isna()].shape

(0, 30)

In [178]:
pr_else3[pr_else3["PositionRank_3P"].isna()].shape

(531, 30)

In [179]:
pr_else3.columns

Index(['PrimaryInstitution', 'OrgName', 'LinkToNext_Year', 'Position',
       'IsElected', 'PositionRank_manual', 'PositionRank_3P',
       'Change_Position', 'Notes', 'InsideGov', 'InstitutionType', 'OrgType',
       'PI_Index', 'OrgRank', 'P1', 'P2', 'P3', 'LinkToNext_PI',
       'LinkToNext_Org', 'LinkToNext_Year_opr', 'Notes_opr', 'L1_Index',
       'L2_Index', 'L3_Index', 'L4_Index', 'L5_Index', 'Change_PI',
       'Change_OrgName', 'Alias_OrgName', '_merge'],
      dtype='object')

In [180]:
# export pr_admin3 for Esther & Jeongsu to review

# export_columns = ["InsideGov","PrimaryInstitution","OrgName","OrgRank","LinkToNext_Year","Position","IsElected","PositionRank_manual","PositionRank_3P","P1","P2","P3","Change_OrgName","Change_Position","Notes"]
# pr_admin3[export_columns].to_excel(path_cleaning + subpath_2_7 + "position_rank_esther_two_ranks.xlsx",index=False)

# 3. match career & pr_admin3 and resolve null Position_3P

#### Resolve. Validation values for resolving null Position_3P

1. OrgName & Position ok. Add Position to P1
2. OrgName & Position ok. Add Position to P2
3. OrgName & Position ok. Add Position to P3
4. Change_OrgName and/or Change_Position
5. Uncertain OrgName
6. Uncertain Position
7. NotJob
8. Multiple CareerSubstring
9. Other - see Notes

In [181]:
careers.head(2)

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,CareerSubstring,OrgString,PrimaryInstitution,OrgName,Position,Notes
0,1996. 3 외교부 10국장,NaN,True,1,1996,3,NaN,외교부,정무원,외교부,국장,NaN
1,유엔 주재 대표부 1등서기관,1975.10,True,1,1975,10,NaN,유엔 주재 대표부,내각,외무성_유엔대표부,1등서기관,NaN


In [182]:
careers_columns = ["CareerString","CareerStartYear","MultipleSubstrings","CareerSubstring","IsJob","PrimaryInstitution","OrgName","Position"]
careers[careers_columns].head(2)

,CareerString,CareerStartYear,MultipleSubstrings,CareerSubstring,IsJob,PrimaryInstitution,OrgName,Position
0,1996. 3 외교부 10국장,1996,1,NaN,True,정무원,외교부,국장
1,유엔 주재 대표부 1등서기관,1975,1,NaN,True,내각,외무성_유엔대표부,1등서기관


In [184]:
pr_else3.head(2)

,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,Change_Position,Notes,InsideGov,InstitutionType,OrgType,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year_opr,Notes_opr,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_PI,Change_OrgName,Alias_OrgName,_merge
0,4.15문화창작단,NaN,Current,단장,NaN,1,NaN,NaN,NaN,0,NaN,NaN,201.0.0.0.0,0,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,0,0,0,0,NaN,NaN,NaN,both
1,4.15문화창작단,NaN,Current,부단장,NaN,2,NaN,NaN,NaN,0,NaN,NaN,201.0.0.0.0,0,NaN,NaN,NaN,NaN,NaN,Current,NaN,201,0,0,0,0,NaN,NaN,NaN,both


In [185]:
pr_else3_incomplete = pr_else3[pr_else3["PositionRank_3P"].isna()]
pr_else3_incomplete.shape

(531, 30)

In [186]:
pr_match_columns = ["InstitutionType","OrgType","PrimaryInstitution","OrgName","LinkToNext_Year","Position","IsElected","PositionRank_manual","PositionRank_3P","P1","P2","P3","Change_PI","Change_OrgName","Change_Position","Notes"]
pr_else3_incomplete = pr_else3_incomplete[pr_match_columns]
pr_else3_incomplete["Resolution"] = np.nan

In [188]:
pr_order_columns = ["InstitutionType","OrgType","PrimaryInstitution","OrgName","LinkToNext_Year","Position","IsElected","PositionRank_manual","PositionRank_3P","P1","P2","P3","Resolution","Change_PI","Change_OrgName","Change_Position","Notes"]
pr_else3_incomplete = pr_else3_incomplete[pr_order_columns]
pr_else3_incomplete.head(2)

,InstitutionType,OrgType,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,P1,P2,P3,Resolution,Change_PI,Change_OrgName,Change_Position,Notes
0,NaN,NaN,4.15문화창작단,NaN,Current,단장,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,4.15문화창작단,NaN,Current,부단장,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
# drop duplicates - as some PI pairs were matched to multiple career strings
pr_else3_incomplete.drop_duplicates(inplace=True)

In [191]:
pr_else3_incomplete.shape

(531, 17)

In [192]:
pr_else3_incomplete[pr_else3_incomplete.duplicated()].shape

(0, 17)

In [193]:
key_columns3 = ["PrimaryInstitution","OrgName","Position"]
career_pos_mismatch = careers[careers_columns].merge(pr_else3_incomplete,on=key_columns3,how="inner",indicator=False)
# merge_results(career_pos_mismatch)

In [194]:
# career_pos_mismatch[career_pos_mismatch["_merge"]=="right_only"]

In [195]:
career_pos_mismatch.shape

(1363, 22)

In [196]:
career_pos_mismatch.head(10)

,CareerString,CareerStartYear,MultipleSubstrings,CareerSubstring,IsJob,PrimaryInstitution,OrgName,Position,InstitutionType,OrgType,LinkToNext_Year,IsElected,PositionRank_manual,PositionRank_3P,P1,P2,P3,Resolution,Change_PI,Change_OrgName,Change_Position,Notes
0,김일성-김정일주의청년동맹 중앙위원회 1비서,2016,1,NaN,True,김일성-김정일주의청년동맹,중앙위원회,1비서,NaN,NaN,2021,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,김일성-김정일주의청년동맹 중앙위원회 1비서 * 전임: 전용남,2017,1,NaN,True,김일성-김정일주의청년동맹,중앙위원회,1비서,NaN,NaN,2021,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1996. 1 김일성사회주의 청년동맹 중앙위 1비서,1996,1,NaN,True,사회주의애국청년동맹,중앙위원회,1비서,당외곽및사회단체_근로단체,NaN,Current,NaN,1,NaN,위원장,부위원장,위원,NaN,NaN,NaN,NaN,NaN
3,2007.12 김일성사회주의청년동맹 중앙위원회 1비서 *전임 : 김경호,2007,1,NaN,True,사회주의애국청년동맹,중앙위원회,1비서,당외곽및사회단체_근로단체,NaN,Current,NaN,1,NaN,위원장,부위원장,위원,NaN,NaN,NaN,NaN,NaN
4,김일성사회주의청년동맹 중앙위원회 1비서,1996,1,NaN,True,김일성사회주의청년동맹,중앙위원회,1비서,NaN,NaN,2016,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2012. 3 김일성사회주의청년동맹 1비서,2012,1,NaN,True,사회주의애국청년동맹,NaN,1비서,당외곽및사회단체_근로단체,NaN,Current,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"1981. 2군단 부군단장, 인민군 대좌",1981,1,NaN,True,총참모부,작전총국_제3처_제2군단,군단장,인민군,NaN,Current,NaN,1,NaN,NaN,NaN,NaN,NaN,인민군,총참모부_작전총국_제3처_제2군단,NaN,NaN
7,군 2군단장,1965,1,NaN,True,총참모부,작전총국_제3처_제2군단,군단장,인민군,NaN,Current,NaN,1,NaN,NaN,NaN,NaN,NaN,인민군,총참모부_작전총국_제3처_제2군단,NaN,NaN
8,1994.10 인민군 2군단장,1994,1,NaN,True,총참모부,작전총국_제3처_제2군단,군단장,인민군,NaN,Current,NaN,1,NaN,NaN,NaN,NaN,NaN,인민군,총참모부_작전총국_제3처_제2군단,NaN,NaN
9,1965. 1 인민군 2군단장,1965,1,NaN,True,총참모부,작전총국_제3처_제2군단,군단장,인민군,NaN,Current,NaN,1,NaN,NaN,NaN,NaN,NaN,인민군,총참모부_작전총국_제3처_제2군단,NaN,NaN


In [197]:
career_pos_mismatch[career_pos_mismatch["Position"].str.lower()=="uncertain"]

,CareerString,CareerStartYear,MultipleSubstrings,CareerSubstring,IsJob,PrimaryInstitution,OrgName,Position,InstitutionType,OrgType,LinkToNext_Year,IsElected,PositionRank_manual,PositionRank_3P,P1,P2,P3,Resolution,Change_PI,Change_OrgName,Change_Position,Notes
15,1966. 호위총국 근무,1966,1,NaN,False,호위사령부,호위총국,Uncertain,인민군,NaN,Current,NaN,?,NaN,NaN,NaN,NaN,NaN,국방위원회,호위사령부_호위총국,NaN,NaN
16,호위총국 근무,1966,1,NaN,True,호위사령부,호위총국,Uncertain,인민군,NaN,Current,NaN,?,NaN,NaN,NaN,NaN,NaN,국방위원회,호위사령부_호위총국,NaN,NaN


In [198]:
career_pos_mismatch.shape

(1363, 22)

In [199]:
# export and edit

career_pos_mismatch.to_excel(path_cleaning + subpath_2_7 + "2.0_mismatch_군및기타_0.xlsx",index=False)